In [1]:
import pandas as pd
import nltk
import numpy as np

import tensorflow as tf

In [16]:
from google.colab import files
uploaded = files.upload()

Saving 5000TravelQuestionsDataset.xlsx to 5000TravelQuestionsDataset (1).xlsx


In [18]:
xlsx_file = io.BytesIO(uploaded.get('5000TravelQuestionsDataset.xlsx'))

In [26]:
col_names = ['questions', 'Label', 'Type']
df = pd.read_excel(xlsx_file, 'Questions',header=None,names=col_names)

In [27]:

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   Label      5000 non-null   object
 2   Type       5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB
None


### Pre processing the dataset


In [28]:
import re

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [29]:
df['clean_questions'] = [clean_text(question) for question in df['questions']]

In [33]:
print(df['clean_questions'][1])
print(df['questions'][1])


what are the companies which organize shark feeding events for scuba divers 
What are the companies which organize shark feeding events for scuba divers?


In [34]:
import string

def remove_punctuation(x):
    return "".join([char for char in x if char not in string.punctuation])

# dataset['questions'] = [remove_punctuation(question) for question in dataset['questions']]
# dataset['questions'][test_id]

In [75]:


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

stop_words = stopwords.words('english')
from nltk import word_tokenize

def remove_stopwords(x):
    words = word_tokenize(x)
    return " ".join([word for pos, word in enumerate(words) if (pos < 1) or (word not in stop_words)])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [41]:
df['questions_stop'] = [remove_punctuation(remove_stopwords(question)) for question in df['questions']]
print(df['questions'][test_id])
print(df['questions_stop'][test_id])
# print(stop_words)

What are the companies which organize shark feeding events for scuba divers?
What companies organize shark feeding events scuba divers 


### **Creating Features**

In [42]:
#Bag of Words using Lemmatization

In [47]:

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
def lemmatize(x):
    filtered_words = nltk.word_tokenize(x)
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]
    return " ".join(lemmatized)

df['lem_questions'] = [lemmatize(question) for question in df['clean_questions']]
df['lem_questions'][test_id]


'what are the company which organize shark feeding event for scuba diver'

In [48]:
#POS tags as features

In [54]:
from nltk import pos_tag
def pos_tagging(x):
    words = nltk.word_tokenize(x)
    lst = [ r[1] for r in pos_tag(words)] 
    return ' '.join(lst)

df['pos_questions'] = [pos_tagging(question) for question in df['questions']]
print(df['pos_questions'][1])
print(df['questions'][1])

WP VBP DT NNS WDT VBP NN NN NNS IN NN NNS .
What are the companies which organize shark feeding events for scuba divers?


In [55]:
#Bigram as Fearutes

In [57]:

def bigram(x):
    new_words = ""
    pre_word = None
    for word in x.strip().split(' '):
        
        if pre_word is not None:
            new_words += "{}{} ".format(pre_word, word)
        pre_word = word
    return new_words[:-1]

# print(bigram("what are the company which organize shark feeding event for scuba divers "))

df['questions_bigram'] = [bigram(question) for question in df['questions_stop']]

print(df['questions'][1])
print(df['questions_bigram'][1])

What are the companies which organize shark feeding events for scuba divers?
Whatcompanies companiesorganize organizeshark sharkfeeding feedingevents eventsscuba scubadivers


Name Entity as Feature

In [79]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)
    
    def remove_null(x):
        if '' in x:
            x.remove('')
        return x

    lst = remove_null(continuous_chunk)
    return ' '.join(lst)

txt = "Barack Obama is a great person." 
print (get_continuous_chunks(txt))



df['ne_questions'] = [get_continuous_chunks(question) for question in df['questions']]
print(df['ne_questions'][1])

Barack Obama



In [83]:
#Head Word as Feature

In [85]:
# Head word tokenizer
import spacy
nlp = spacy.load("en_core_web_sm")
def head_word_tokenizer(text):
    head_words = []
    for token in nlp(text):
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass":
            head_words.append(token.text)
#             head_words.append(token.head.text)
    return head_words

In [86]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

head_words_vectorizer = CountVectorizer(tokenizer = head_word_tokenizer,max_features=100,stop_words=stopwords.words('english'))
head_words_vector = head_words_vectorizer.fit_transform(df["questions"].values).toarray()

In [117]:
head_words_vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [80]:
#Count vecotrizer (Remove stop words and vectorize)

In [82]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

def get_count_vect(documents):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)#, stop_words=stopwords.words('english')
    X = vectorizer.fit_transform(documents).toarray()
    print(X.shape)
    return X

print(get_count_vect(df['questions']))

(5000, 1177)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


**Model Details**

Model Metrics


In [58]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def PRC_matrics(y_test, prediction):
    # calculate prediction
    precision = precision_score(y_test, prediction, labels=[1,2], average='micro')*100
    print('Precision: %.3f' % precision)

    # calculate recall
    recall = recall_score(y_test, prediction, labels=[1,2], average='micro')*100
    print('Recall: %.3f' % recall)
    
    # calculate score
#     score = f1_score(y_test, prediction, average='micro')
    f1_score = 2 * (precision * recall) / (precision + recall)
    print('F-Measure: %.3f' % f1_score)
    
    
    acc = accuracy_score(y_test, prediction)*100
#     tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
#     acc2 = (tn + tp)*100/(tn + fp + fn + tp)
    print('Accuracy score: %.3f' % acc)
    
    
    cm = confusion_matrix(y_test, prediction)
    print("\nConfustion matrix: \n{}".format(cm))
    
    return precision, recall, f1_score, acc

In [63]:
#Geeting Y values 

def get_encoded_y(feature_set='Label'):
    le = LabelEncoder()
    y = le.fit_transform(df[feature_set])
    return y

SVM model details

In [60]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


def train_with_svm(XX, y):
    best_prediction = None
    best_test = None
    best_accuracy = 0
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(XX):
        fold += 1
        X_train, X_test = XX[train_index], XX[test_index]
        y_train, y_test = y[train_index], y[test_index]

        SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
        SVM.fit(X_train,y_train)
        predictions_SVM1 = SVM.predict(X_test)
        acc = accuracy_score(predictions_SVM1, y_test)*100
        if best_accuracy < acc:
            best_accuracy = acc
            best_prediction = predictions_SVM1
            best_test = y_test
            best_model = SVM
        accuracies.append(acc)
        print("Fold - {} - {} - {:.2f}".format(fold, "SVM Accuracy Score -> ",acc))

    print("Mean Accuracy {:.2f} \nStd Accuracy {:.2f}\n\n".format(np.mean(accuracies), np.std(accuracies)))
    
    print("Best accuracy : {}".format(best_accuracy))
    PRC_matrics(best_test, best_prediction)

Task 1 - A traditional ML classifier s.a. SVM or Logistic Regression with at least 5 of the features mentioned in the paper.

In [93]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix, csr_matrix, hstack


y = get_encoded_y('Label')

X_lem = df['lem_questions']
tfidf_lem = TfidfVectorizer(max_features=5000)
tfidf_lem.fit(X_lem)

X_pos = df['pos_questions']

X_bigram = df['questions_bigram']

X_ne = df['ne_questions']

XX = csr_matrix(hstack([tfidf_lem.transform(X_lem) ,get_count_vect(X_pos),get_count_vect(X_ne), get_count_vect(X_bigram),head_words_vector]))
XX.shape

(5000, 27)
(5000, 326)
(5000, 527)


(5000, 5980)

Feature Selection

In [95]:
#Select K best features

from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, k=3000).fit_transform(XX, y)
X_new.shape

(5000, 3000)

In [97]:
#Remove features with low variance with target 

from sklearn.feature_selection import VarianceThreshold
X_new_var = VarianceThreshold(threshold=(0.01)).fit_transform(XX)
X_new_var.shape
# selector.fit_transform(features)

(5000, 53)

Training the model with SVM - Using all features

In [99]:
train_with_svm(XX, get_encoded_y('Label'))

Fold - 1 - SVM Accuracy Score ->  - 75.60
Fold - 2 - SVM Accuracy Score ->  - 79.60
Fold - 3 - SVM Accuracy Score ->  - 78.00
Fold - 4 - SVM Accuracy Score ->  - 77.80
Fold - 5 - SVM Accuracy Score ->  - 79.40
Fold - 6 - SVM Accuracy Score ->  - 80.60
Fold - 7 - SVM Accuracy Score ->  - 80.00
Fold - 8 - SVM Accuracy Score ->  - 77.80
Fold - 9 - SVM Accuracy Score ->  - 78.60
Fold - 10 - SVM Accuracy Score ->  - 75.40
Mean Accuracy 78.28 
Std Accuracy 1.66


Best accuracy : 80.60000000000001
Precision: 75.949
Recall: 82.192
F-Measure: 78.947
Accuracy score: 80.600

Confustion matrix: 
[[ 52   0   0   2   1   4   0]
 [  1   8   0   0   0   5   0]
 [  4   0  43   0   1   2   0]
 [  5   0   3 112  10  18   0]
 [  1   1   1   8  95   3   1]
 [  5   2   2  14   2  76   0]
 [  0   0   0   1   0   0  17]]


## **Word Embeddings as Features**

Gensim Doc2Vec - Using Sentence Vectors

In [105]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text

card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df.clean_questions)]

#display the tagged docs


In [107]:
#instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model.build_vocab(card_docs)
#train model
model.train(card_docs, total_examples=model.corpus_count
            , epochs=model.epochs)

In [110]:
#generate vectors
card2vec = [model.infer_vector((df['clean_questions'][i].split(' '))) 
            for i in range(0,len(df['clean_questions']))]


In [128]:
import numpy as np
#Create a list of lists
dtv= np.array(card2vec)
#set list to dataframe column


,questions,Label,Type,clean_questions,questions_stop,lem_questions,pos_questions,questions_bigram,ne_questions,card2vec
0,What are the special things we (husband and me...,TTD,TTDSIG,what are the special things we husband and me...,What special things husband 5 day stay Cape ...,what are the special thing we husband and me c...,WP VBP DT JJ NNS PRP ( NN CC PRP ) MD VB IN DT...,Whatspecial specialthings things husband husba...,Cape Town,0.140274
1,What are the companies which organize shark fe...,TTD,TTDOTH,what are the companies which organize shark fe...,What companies organize shark feeding events s...,what are the company which organize shark feed...,WP VBP DT NNS WDT VBP NN NN NNS IN NN NNS .,Whatcompanies companiesorganize organizeshark ...,,0.001748


In [129]:
train_with_svm(dtv, get_encoded_y('Label'))

Fold - 1 - SVM Accuracy Score ->  - 56.40
Fold - 2 - SVM Accuracy Score ->  - 55.60
Fold - 3 - SVM Accuracy Score ->  - 57.20
Fold - 4 - SVM Accuracy Score ->  - 57.00
Fold - 5 - SVM Accuracy Score ->  - 57.60
Fold - 6 - SVM Accuracy Score ->  - 59.60
Fold - 7 - SVM Accuracy Score ->  - 56.60
Fold - 8 - SVM Accuracy Score ->  - 55.00
Fold - 9 - SVM Accuracy Score ->  - 56.40
Fold - 10 - SVM Accuracy Score ->  - 57.40
Mean Accuracy 56.88 
Std Accuracy 1.18


Best accuracy : 59.599999999999994
Precision: 52.113
Recall: 50.685
F-Measure: 51.389
Accuracy score: 59.600

Confustion matrix: 
[[37  0  1  5  3 13  0]
 [ 2  0  4  2  0  6  0]
 [ 8  1 21  2  0 18  0]
 [ 8  0  6 85 24 25  0]
 [ 6  0  0 15 76 10  3]
 [ 7  2  4 14  7 66  1]
 [ 0  0  0  1  0  4 13]]


# **LSTM classification**




In [130]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 25
# This is fixed.
EMBEDDING_DIM = 160
epochs = 10
batch_size = 64

In [131]:
from tensorflow.keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(df['questions'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5582 unique tokens.


In [134]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(df['questions'].values)
print(X[0])
X_lstm = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_lstm.shape)


def get_dummi_y(_type='a'):
    y = pd.get_dummies(df[_type]).values
    print('Shape of label tensor:', y.shape)
    return y

[4, 7, 2, 321, 105, 31, 1837, 17, 68, 9, 20, 71, 6, 194, 48, 32, 22, 376, 111]
Shape of data tensor: (5000, 25)


In [136]:
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import Sequential

def get_lstm_model(X, y, verbose=0):
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
    # model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if verbose == 1:
        print(model.summary())
    return model

get_lstm_model(X_lstm, get_dummi_y('Type'), 1)

Shape of label tensor: (5000, 79)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 160)           800000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 25, 160)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               279888    
_________________________________________________________________
dense (Dense)                (None, 79)                15563     
Total params: 1,095,451
Trainable params: 1,095,451
Non-trainable params: 0
_________________________________________________________________
None


In [138]:

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

epochs = 10
batch_size = 64

def train_LSTM(X, y):
    best_prediction = None
    best_test = None
    best_accuracy = 0

    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(X):
        fold += 1
        print("FOLD {}".format(fold))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = get_lstm_model(X, y, 0)

        hist = model.fit(X_train, y_train, verbose=0, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
        predictions = model.predict(X_test)
        for item in hist.history.items():
            mean_val = np.mean(item[1])
            if item[0] == 'accuracy':
                acc = mean_val
            print("Mean {} : {}".format(item[0], mean_val))
        
        
        print("\n")
        if best_accuracy < acc:
            best_accuracy = acc
            best_prediction = predictions
            best_test = y_test
            
    fine_pred = [np.argmax(p) for p in best_prediction]
    fine_gt = [np.argmax(p) for p in best_test]
    PRC_matrics(fine_pred, fine_gt)

In [139]:

train_LSTM(X_lstm, get_dummi_y('Label'))

Shape of label tensor: (5000, 10)
FOLD 1
Mean loss : 0.49615826995836365
Mean accuracy : 0.823539094792472
Mean val_loss : 1.0179834100935194
Mean val_accuracy : 0.6612345708741082


FOLD 2
Mean loss : 0.573581064119935
Mean accuracy : 0.7948148250579834
Mean val_loss : 1.0652254968881607
Mean val_accuracy : 0.6450000070035458


FOLD 3
Mean loss : 0.544838204048574
Mean accuracy : 0.8068209849298
Mean val_loss : 0.9779676422476768
Mean val_accuracy : 0.6666666623204947


FOLD 4
Mean loss : 0.45404748655855653
Mean accuracy : 0.8368395060300827
Mean val_loss : 1.1208764433860778
Mean val_accuracy : 0.64266666918993


FOLD 5
Mean loss : 0.5250205309130251
Mean accuracy : 0.8148765452206135
Mean val_loss : 1.003645971417427
Mean val_accuracy : 0.672500004991889


FOLD 6
Mean loss : 0.6315906611936433
Mean accuracy : 0.7734038744653974
Mean val_loss : 1.0543848190988814
Mean val_accuracy : 0.646349202309336


FOLD 7
Mean loss : 0.4952920248938931
Mean accuracy : 0.82463648253017
Mean val_l